In [61]:
import pandas as pd
import os.path as osp
import os

In [30]:
all_models=["resnet18", "resnet34", "xception"]
all_datasets=['cifar10']
all_optim = [('adam', "0.001"), ('sgd', "0.1")]
all_initialization = ['pretrain', 'kaiming_normal']
all_lr_scheduler = ['reduceLR', 'none', 'cosine_annealingLR']
all_noise_injection = ["0.0", "0.03", "0.07", "0.13"]
all_noise_sparsity = ["0.0", "0.2", "0.4", "0.6"]
all_lr_rate = ['0.001', '0.1']
all_phase = ['train', 'validation']
all_epochs = 5
all_folds = 3
device = 'cuda:0'

In [56]:
def generate_all_examins():
    df = pd.DataFrame()
    for dataset in all_datasets:
        for model in all_models:
            for optim, lr in all_optim:
                for initialization in all_initialization:
                    for lr_scheduler in all_lr_scheduler:
                        for noise_injection in all_noise_injection:
                            for noise_sparsity in all_noise_sparsity:
                                if noise_injection == "0.0" and noise_sparsity !="0.0":
                                    continue
                                df = df._append({
                                    "dataset": dataset, "model": model,
                                    "optim": optim, 
                                    "init": initialization, "lr_scheduler": lr_scheduler,
                                    "np": f"np={noise_injection}", "ns": f"ns={noise_sparsity}", "lr": f"lr={lr}",
                                    "folds": all_folds, "epochs": all_epochs,
                                    "done": False
                                    }, ignore_index=True)
    return df

In [112]:
df = generate_all_examins()
df.to_csv("logs/examins.csv", index_label="index")

In [113]:
base_path = "/home/vision/Repo/cleanset/logs"
def check_wich_examines_done(path="logs/examins.csv"):
    df = pd.read_csv(path, index_col='index')
    cols = [ "dataset", "model", "optim", "init", "lr_scheduler", "np", "ns", "lr" ]
    # df = df[df['model'] == 'xception']
    for index, row in df.iterrows():
        examin_dir = osp.join(base_path, *[str(row[col]) for col in cols])
        if osp.isdir(examin_dir):
            number_of_folds = len(os.listdir(examin_dir))
            if number_of_folds == all_folds:
                df.loc[index, 'done'] = True
    return df


In [114]:
df = check_wich_examines_done()
df.to_csv("logs/examins.csv")